In [1]:
from __future__ import print_function
import argparse
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable
import torch.nn.functional as F
import numpy as np
from PIL import Image
from glob import glob
from matplotlib import pyplot as plt

In [2]:
parser = argparse.ArgumentParser()
parser.add_argument('--dataset', required=True, help='cifar10 | lsun | imagenet | folder | lfw ')
parser.add_argument('--dataroot', required=True, help='path to dataset')
parser.add_argument('--workers', type=int, help='number of data loading workers', default=2)
parser.add_argument('--batch_size', type=int, default=16, help='input batch size')
parser.add_argument('--image_size', type=int, default=64, help='the height / width of the input image to network')
parser.add_argument('--nz', type=int, default=64, help='size of the latent z vector')
parser.add_argument('--niter', type=int, default=25, help='number of epochs to train for')
parser.add_argument('--lr', type=float, default=0.0001, help='learning rate, default=0.0002')
parser.add_argument('--beta1', type=float, default=0.4, help='beta1 for adam. default=0.5')
parser.add_argument('--cuda', action='store_true', help='enables cuda')
parser.add_argument('--netD', default='', help="path to netD (to continue training)")
parser.add_argument('--netG', default='', help="path to netG (to continue training)")
parser.add_argument('--base_dir', default='./train7/', help='directory to save image/model/data')
parser.add_argument('--k_init', type=float, default=0.0, help='initial value of k')
parser.add_argument('--gamma', type=float, default=0.4, help='diversity ratio')
parser.add_argument('--numOfKernel', type=int, default=64, help='Number of Kernels in G\'s Conv')
parser.add_argument('--lr_decay', type=float, default=0.95, help='learning rate decay rate')
parser.add_argument('--lr_decay_every', type=int, default=3000, help='decay lr how many iterations')

_StoreAction(option_strings=['--lr_decay_every'], dest='lr_decay_every', nargs=None, const=None, default=3000, type=<type 'int'>, choices=None, help='decay lr how many iterations', metavar=None)

In [3]:
opt = parser.parse_args(['--dataset', 'folder', '--dataroot', '/home/ubuntu/data/work/', '--cuda', '--niter', '25'])
print(opt)

Namespace(base_dir='./train7/', batch_size=16, beta1=0.4, cuda=True, dataroot='/home/ubuntu/data/work/', dataset='folder', gamma=0.4, image_size=64, k_init=0.0, lr=0.0001, lr_decay=0.95, lr_decay_every=3000, netD='', netG='', niter=25, numOfKernel=64, nz=64, workers=2)


In [4]:
try:
    os.makedirs(opt.base_dir)
except OSError:
    pass

In [5]:
# if opt.manualSeed is None:
#     opt.manualSeed = random.randint(1, 10000)
# print("Random Seed: ", opt.manualSeed)
# random.seed(opt.manualSeed)
# torch.manual_seed(opt.manualSeed)
# if opt.cuda:
#     torch.cuda.manual_seed_all(opt.manualSeed)

In [6]:
cudnn.benchmark = True
if torch.cuda.is_available() and not opt.cuda:
    print("WARNING: You have a CUDA device, so you should probably run with --cuda")

In [7]:
torch.cuda.set_device(0)

In [8]:
file_list = glob('/home/ubuntu/data/work/img_align_celeba/*')
mini_batch = len(file_list) / opt.batch_size
def get_permutation():
    return np.random.permutation(len(file_list))
def get_samples(index, i):
    data = torch.FloatTensor(opt.batch_size, 3, opt.image_size, opt.image_size)
    j = 0
    for i in range(opt.batch_size*i, opt.batch_size*(i+1)):
        data[j] = transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))(transforms.ToTensor()(Image.open(file_list[index[i]])))
        j += 1
    return data

In [9]:
ngpu = 1
nz = int(opt.nz)
k = opt.k_init

In [10]:
# custom weights initialization called on netG and netD
def G_weights_init(m):
    classname = m.__class__.__name__

    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.002)

    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

def D_weights_init(m):
    classname = m.__class__.__name__

    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)

    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

In [11]:
class G_CNN(nn.Module):
    def __init__(self, embedding_size, kernel_num):
        super(G_CNN, self).__init__()
        self.kernel_num = kernel_num
        FC_start_size = 8 * 8 * self.kernel_num
        self.start = nn.Linear(embedding_size, FC_start_size)
        #all layers
        self.main_arch = nn.Sequential(
            #repeated units
            nn.Conv2d(in_channels=kernel_num, out_channels=kernel_num, kernel_size=3, padding=1),
            nn.ELU(inplace=True),
            nn.Conv2d(in_channels=kernel_num, out_channels=kernel_num, kernel_size=3, padding=1),
            nn.ELU(inplace=True),
            nn.UpsamplingNearest2d(scale_factor=2),
            #repeated units
            nn.Conv2d(in_channels=kernel_num, out_channels=kernel_num, kernel_size=3, padding=1),
            nn.ELU(inplace=True),
            nn.Conv2d(in_channels=kernel_num, out_channels=kernel_num, kernel_size=3, padding=1),
            nn.ELU(inplace=True),
            nn.UpsamplingNearest2d(scale_factor=2),
            #repeated units
            nn.Conv2d(in_channels=kernel_num, out_channels=kernel_num, kernel_size=3, padding=1),
            nn.ELU(inplace=True),
            nn.Conv2d(in_channels=kernel_num, out_channels=kernel_num, kernel_size=3, padding=1),
            nn.ELU(inplace=True),
            nn.UpsamplingNearest2d(scale_factor=2),
            #last layer
            #nn.Conv2d(in_channels=kernel_num, out_channels=3, kernel_size=3, padding=1)
            nn.Conv2d(in_channels=kernel_num, out_channels=kernel_num, kernel_size=3, padding=1),
            nn.ELU(inplace=True),
            nn.Conv2d(in_channels=kernel_num, out_channels=kernel_num, kernel_size=3, padding=1),
            nn.ELU(inplace=True),
            nn.Conv2d(in_channels=kernel_num, out_channels=3, kernel_size=3, padding=1),
            nn.Tanh()
        )

    def forward(self, in_data):
        in_data = self.start(in_data.view(-1, 64))
        in_data = in_data.view(-1, self.kernel_num, 8, 8)
        output = self.main_arch(in_data)
        return output

In [12]:
netG = G_CNN(embedding_size=nz, kernel_num=opt.numOfKernel)
netG.apply(G_weights_init)
if opt.netG != '':
    netG.load_state_dict(torch.load(opt.netG))

In [13]:
#Discriminator
class D_CNN(nn.Module):
    def __init__(self, embedding_size, base_kernel_num):
        super(D_CNN, self).__init__()
        self.base_kernel_num = base_kernel_num
        
        #encoder w/o flatten and FC
        self.encoder_wo_FC = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=base_kernel_num, kernel_size=3, padding=1),
            nn.ELU(inplace=True),
            
            nn.Conv2d(in_channels=base_kernel_num, out_channels=base_kernel_num, kernel_size=3, padding=1),
            nn.ELU(inplace=True),
            nn.Conv2d(in_channels=base_kernel_num, out_channels=base_kernel_num, kernel_size=3, padding=1),
            nn.ELU(inplace=True),
            #downsampling
            nn.Conv2d(in_channels=base_kernel_num, out_channels=2*base_kernel_num, kernel_size=1, padding=0, stride=1),
            nn.AvgPool2d(kernel_size=2,stride=2),
            
            nn.Conv2d(in_channels=2*base_kernel_num, out_channels=2*base_kernel_num, kernel_size=3, padding=1),
            nn.ELU(inplace=True),
            nn.Conv2d(in_channels=2*base_kernel_num, out_channels=2*base_kernel_num, kernel_size=3, padding=1),
            nn.ELU(inplace=True),
            #downsampling
            nn.Conv2d(in_channels=2*base_kernel_num, out_channels=3*base_kernel_num, kernel_size=1, padding=0, stride=1),
            nn.AvgPool2d(kernel_size=2,stride=2),
            
            nn.Conv2d(in_channels=3*base_kernel_num, out_channels=3*base_kernel_num, kernel_size=3, padding=1),
            nn.ELU(inplace=True),
            nn.Conv2d(in_channels=3*base_kernel_num, out_channels=3*base_kernel_num, kernel_size=3, padding=1),
            nn.ELU(inplace=True),
            #downsampling
            nn.Conv2d(in_channels=3*base_kernel_num, out_channels=4*base_kernel_num, kernel_size=1, padding=0, stride=1),
            nn.AvgPool2d(kernel_size=2,stride=2),
            
            nn.Conv2d(in_channels=4*base_kernel_num, out_channels=4*base_kernel_num, kernel_size=3, padding=1),
            nn.ELU(inplace=True),
            nn.Conv2d(in_channels=4*base_kernel_num, out_channels=4*base_kernel_num, kernel_size=3, padding=1),
            nn.ELU(inplace=True)
        )
        FC_mid1_size = 8 * 8 * 4 * self.base_kernel_num
        FC_mid2_size = 8 * 8 * self.base_kernel_num
        self.FC1 = nn.Linear(FC_mid1_size, embedding_size)
        self.FC2 = nn.Linear(embedding_size, FC_mid2_size)
        self.decoder_wo_FC = nn.Sequential(
            #repeated units
            nn.Conv2d(in_channels=base_kernel_num, out_channels=base_kernel_num, kernel_size=3, padding=1),
            nn.ELU(inplace=True),
            nn.Conv2d(in_channels=base_kernel_num, out_channels=base_kernel_num, kernel_size=3, padding=1),
            nn.ELU(inplace=True),
            nn.UpsamplingNearest2d(scale_factor=2),
            #repeated units
            nn.Conv2d(in_channels=base_kernel_num, out_channels=base_kernel_num, kernel_size=3, padding=1),
            nn.ELU(inplace=True),
            nn.Conv2d(in_channels=base_kernel_num, out_channels=base_kernel_num, kernel_size=3, padding=1),
            nn.ELU(inplace=True),
            nn.UpsamplingNearest2d(scale_factor=2),
            #repeated units
            nn.Conv2d(in_channels=base_kernel_num, out_channels=base_kernel_num, kernel_size=3, padding=1),
            nn.ELU(inplace=True),
            nn.Conv2d(in_channels=base_kernel_num, out_channels=base_kernel_num, kernel_size=3, padding=1),
            nn.ELU(inplace=True),
            nn.UpsamplingNearest2d(scale_factor=2),
            #last layer
            #nn.Conv2d(in_channels=base_kernel_num, out_channels=3, kernel_size=3, padding=1)
            nn.Conv2d(in_channels=base_kernel_num, out_channels=base_kernel_num, kernel_size=3, padding=1),
            nn.ELU(inplace=True),
            nn.Conv2d(in_channels=base_kernel_num, out_channels=base_kernel_num, kernel_size=3, padding=1),
            nn.ELU(inplace=True),
            nn.Conv2d(in_channels=base_kernel_num, out_channels=3, kernel_size=3, padding=1)
        )

    def forward(self, in_data):
        in_data = self.encoder_wo_FC(in_data)
        
        in_data = in_data.view(-1, 4*self.base_kernel_num * 8 * 8);
        in_data = self.FC1(in_data)
        in_data = self.FC2(in_data)
        in_data = in_data.view(-1, self.base_kernel_num, 8, 8)
        
        output = self.decoder_wo_FC(in_data)
        return output

In [14]:
netD = D_CNN(embedding_size=nz, base_kernel_num=opt.numOfKernel)
netD.apply(D_weights_init)
if opt.netD != '':
    netD.load_state_dict(torch.load(opt.netD))

In [15]:
criterion_L1 = nn.L1Loss()

input = torch.FloatTensor(opt.batch_size, 3, opt.image_size, opt.image_size)
noise = torch.FloatTensor(opt.batch_size, nz)
fixed_noise = torch.FloatTensor(opt.batch_size, nz, 1, 1).uniform_(-1, 1)

if opt.cuda:
    netD.cuda()
    netG.cuda()
    criterion_L1.cuda()
    input = input.cuda()
    noise, fixed_noise = noise.cuda(), fixed_noise.cuda()

input = Variable(input)
noise = Variable(noise)
fixed_noise = Variable(fixed_noise)
np.save(opt.base_dir+'fixed_noise.npy', fixed_noise.data.cpu().numpy())
# setup optimizer
optimizerD = optim.Adam(netD.parameters(), lr=opt.lr, betas=(0.5, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=opt.lr, betas=(0.5, 0.999))

In [ ]:
def adjust_lr(optimizer, niter):
    opt.lr = opt.lr * (0.95 ** (niter // opt.lr_decay_every))
    for param_group in optimizer.param_groups:
        param_group['lr'] = opt.lr
    return optimizer

In [ ]:
data_loss_D = []
data_loss_G = []
data_loss_D_real = []
data_loss_k = []
data_loss_global = []
for epoch in range(opt.niter):
    index = get_permutation()
    for i in range(int(mini_batch)):

        ############################
        # (1) Update D network
        ###########################
        netD.zero_grad()

        # prepare real
        input.data.copy_(get_samples(index, i))
#         real_cpu, _ = data
#         batch_size = real_cpu.size(0)
#         input.data.resize_(real_cpu.size()).copy_(real_cpu)

        # train with real
        output = netD(input)
        errD_real = torch.mean(torch.abs(output - input))  # score on real
        errD_real.backward()  # backward on score on real
        L_x = errD_real.data[0]  # score fore supervision
        
        # generate fake
        #noise.data.resize_(batch_size, nz, 1, 1)
        noise.data.uniform_(-1, 1)
        fake = netG(noise)
        
        # train with fake
        output = netD(fake.detach())
        errD_fake = torch.mean(torch.abs(output - fake))  # score on fake
        errD_fake_use = - k * errD_fake
        errD_fake_use.backward()  # backward on score on fake
        optimizerD.step()
        
        #D_G_z1 = errD_fake.data.mean()  # score fore supervision <- generated when calc D loss
        errD = errD_real + errD_fake_use  # score fore supervision

        

        ############################
        # (2) Update G network
        ############################

        netG.zero_grad()

        # generate fake
#         noise.data.resize_(batch_size, nz, 1, 1)
#         noise.data.uniform_(-1, 1)
        fake_new = netG(noise)
        
        # NOT reuse generated fake samples
        output = netD(fake_new)
        errG = torch.mean(torch.abs(output - fake_new))  # L1
        errG.backward()
        
        L_G = errG.data[0] # score fore supervision <- generated when calc G loss

        optimizerG.step()
        
        # K STEP
        #left -> real
#         output = netD(input)
#         errD_real_k = criterion_L1(output, input)  # score on real
#         #right -> fake
#         fake = netG(noise)
#         output = netD(fake.detach())
#         errD_fake_k = criterion_L1(output, fake.detach())  # score on fake
        
        #Convergence Measure
        cm = L_x + abs(opt.gamma * L_x - L_G)
        
        k += 0.001 * (opt.gamma * L_x - L_G)
        k = max(min(k, 1), 0)
        ############################
        # (3) Report & 100 Batch checkpoint
        ############################
        data_loss_G.append(L_G)
        data_loss_D.append(errD.data[0])
        data_loss_D_real.append(L_x)
        data_loss_k.append(k)
        data_loss_global.append(cm)
        
        if (i + epoch * int(mini_batch)) % opt.lr_decay_every == (opt.lr_decay_every - 1):
            optimizerD = optim.Adam(netD.parameters(), lr=opt.lr*opt.lr_decay, betas=(0.5, 0.999))
            optimizerG = optim.Adam(netG.parameters(), lr=opt.lr*opt.lr_decay, betas=(0.5, 0.999))
            opt.lr = opt.lr * opt.lr_decay
        if i % 10 == 0:
            print('[%d/%d][%d/%d] Loss_D: %.4f Loss_G: %.4f Convergence: %.4f k= %.6f lr = %.7f' % (epoch, opt.niter, i, int(mini_batch), errD.data[0], L_G, cm, k, opt.lr))
        
        if i % 100 == 0:
            fake_fixed = netG(fixed_noise)
            vutils.save_image(fake_fixed.data, opt.base_dir+'%05d_fake_samples_epoch_%03d.png' % (i, epoch), normalize = True, scale_each=True)

    # do checkpointing
    torch.save(netG.state_dict(), opt.base_dir+'netG_epoch_%d.pth' % (epoch))
    torch.save(netD.state_dict(), opt.base_dir+'netD_epoch_%d.pth' % (epoch))
    np.save(opt.base_dir+'k.npy', np.array(k))
    np.save(opt.base_dir+'lr.npy', np.array(opt.lr))

[0/25][0/9765] Loss_D: 0.4499 Loss_G: 0.0472 Convergence: 0.5827 k= 0.000133 lr = 0.0001000
[0/25][10/9765] Loss_D: 0.4492 Loss_G: 0.1245 Convergence: 0.5045 k= 0.001409 lr = 0.0001000
[0/25][20/9765] Loss_D: 0.3714 Loss_G: 0.2301 Convergence: 0.4531 k= 0.001598 lr = 0.0001000
[0/25][30/9765] Loss_D: 0.3718 Loss_G: 0.2300 Convergence: 0.4532 k= 0.000500 lr = 0.0001000
[0/25][40/9765] Loss_D: 0.3659 Loss_G: 0.2953 Convergence: 0.5148 k= 0.000000 lr = 0.0001000
[0/25][50/9765] Loss_D: 0.3563 Loss_G: 0.2666 Convergence: 0.4803 k= 0.000000 lr = 0.0001000
[0/25][60/9765] Loss_D: 0.3488 Loss_G: 0.3471 Convergence: 0.5563 k= 0.000000 lr = 0.0001000
[0/25][70/9765] Loss_D: 0.3797 Loss_G: 0.1578 Convergence: 0.3856 k= 0.000073 lr = 0.0001000
[0/25][80/9765] Loss_D: 0.2946 Loss_G: 0.0998 Convergence: 0.3127 k= 0.000250 lr = 0.0001000
[0/25][90/9765] Loss_D: 0.2789 Loss_G: 0.0559 Convergence: 0.3346 k= 0.000673 lr = 0.0001000
[0/25][100/9765] Loss_D: 0.2594 Loss_G: 0.0556 Convergence: 0.3077 k= 0

[0/25][880/9765] Loss_D: 0.1676 Loss_G: 0.0648 Convergence: 0.1699 k= 0.001379 lr = 0.0001000
[0/25][890/9765] Loss_D: 0.1742 Loss_G: 0.0745 Convergence: 0.1790 k= 0.001313 lr = 0.0001000
[0/25][900/9765] Loss_D: 0.1569 Loss_G: 0.0579 Convergence: 0.1618 k= 0.001369 lr = 0.0001000
[0/25][910/9765] Loss_D: 0.1884 Loss_G: 0.1524 Convergence: 0.2655 k= 0.001277 lr = 0.0001000
[0/25][920/9765] Loss_D: 0.1710 Loss_G: 0.0829 Convergence: 0.1855 k= 0.001172 lr = 0.0001000
[0/25][930/9765] Loss_D: 0.1802 Loss_G: 0.1041 Convergence: 0.2122 k= 0.001117 lr = 0.0001000
[0/25][940/9765] Loss_D: 0.1758 Loss_G: 0.0766 Convergence: 0.1822 k= 0.001134 lr = 0.0001000
[0/25][950/9765] Loss_D: 0.1750 Loss_G: 0.1015 Convergence: 0.2065 k= 0.001028 lr = 0.0001000
[0/25][960/9765] Loss_D: 0.1727 Loss_G: 0.0606 Convergence: 0.1812 k= 0.001042 lr = 0.0001000
[0/25][970/9765] Loss_D: 0.1747 Loss_G: 0.0918 Convergence: 0.1967 k= 0.001050 lr = 0.0001000
[0/25][980/9765] Loss_D: 0.1992 Loss_G: 0.1276 Convergence: 

[0/25][1750/9765] Loss_D: 0.1790 Loss_G: 0.0448 Convergence: 0.2064 k= 0.009877 lr = 0.0001000
[0/25][1760/9765] Loss_D: 0.1633 Loss_G: 0.0642 Convergence: 0.1653 k= 0.009951 lr = 0.0001000
[0/25][1770/9765] Loss_D: 0.1539 Loss_G: 0.0678 Convergence: 0.1604 k= 0.010049 lr = 0.0001000
[0/25][1780/9765] Loss_D: 0.1535 Loss_G: 0.0515 Convergence: 0.1642 k= 0.010215 lr = 0.0001000
[0/25][1790/9765] Loss_D: 0.1543 Loss_G: 0.0435 Convergence: 0.1733 k= 0.010382 lr = 0.0001000
[0/25][1800/9765] Loss_D: 0.1497 Loss_G: 0.0488 Convergence: 0.1614 k= 0.010496 lr = 0.0001000
[0/25][1810/9765] Loss_D: 0.1773 Loss_G: 0.0551 Convergence: 0.1940 k= 0.010597 lr = 0.0001000
[0/25][1820/9765] Loss_D: 0.1655 Loss_G: 0.0467 Convergence: 0.1857 k= 0.010696 lr = 0.0001000
[0/25][1830/9765] Loss_D: 0.1600 Loss_G: 0.0475 Convergence: 0.1772 k= 0.010846 lr = 0.0001000
[0/25][1840/9765] Loss_D: 0.1577 Loss_G: 0.0590 Convergence: 0.1625 k= 0.010922 lr = 0.0001000
[0/25][1850/9765] Loss_D: 0.1722 Loss_G: 0.0909 Co

[0/25][2620/9765] Loss_D: 0.1443 Loss_G: 0.0497 Convergence: 0.1534 k= 0.017605 lr = 0.0001000
[0/25][2630/9765] Loss_D: 0.1425 Loss_G: 0.0523 Convergence: 0.1487 k= 0.017726 lr = 0.0001000
[0/25][2640/9765] Loss_D: 0.1569 Loss_G: 0.0607 Convergence: 0.1602 k= 0.017745 lr = 0.0001000
[0/25][2650/9765] Loss_D: 0.1501 Loss_G: 0.0446 Convergence: 0.1673 k= 0.017721 lr = 0.0001000
[0/25][2660/9765] Loss_D: 0.1570 Loss_G: 0.0735 Convergence: 0.1684 k= 0.017695 lr = 0.0001000
[0/25][2670/9765] Loss_D: 0.1473 Loss_G: 0.0499 Convergence: 0.1577 k= 0.017706 lr = 0.0001000
[0/25][2680/9765] Loss_D: 0.1572 Loss_G: 0.0520 Convergence: 0.1694 k= 0.017781 lr = 0.0001000
[0/25][2690/9765] Loss_D: 0.1471 Loss_G: 0.0527 Convergence: 0.1547 k= 0.017830 lr = 0.0001000
[0/25][2700/9765] Loss_D: 0.1448 Loss_G: 0.0417 Convergence: 0.1623 k= 0.017892 lr = 0.0001000
[0/25][2710/9765] Loss_D: 0.1548 Loss_G: 0.0551 Convergence: 0.1628 k= 0.017954 lr = 0.0001000
[0/25][2720/9765] Loss_D: 0.1502 Loss_G: 0.0612 Co

[0/25][3490/9765] Loss_D: 0.1539 Loss_G: 0.0435 Convergence: 0.1734 k= 0.020513 lr = 0.0000950
[0/25][3500/9765] Loss_D: 0.1392 Loss_G: 0.0682 Convergence: 0.1522 k= 0.020609 lr = 0.0000950
[0/25][3510/9765] Loss_D: 0.1374 Loss_G: 0.0574 Convergence: 0.1404 k= 0.020674 lr = 0.0000950
[0/25][3520/9765] Loss_D: 0.1393 Loss_G: 0.0448 Convergence: 0.1518 k= 0.020672 lr = 0.0000950
[0/25][3530/9765] Loss_D: 0.1599 Loss_G: 0.0619 Convergence: 0.1633 k= 0.020767 lr = 0.0000950
[0/25][3540/9765] Loss_D: 0.1396 Loss_G: 0.0435 Convergence: 0.1533 k= 0.020849 lr = 0.0000950
[0/25][3550/9765] Loss_D: 0.1433 Loss_G: 0.0424 Convergence: 0.1597 k= 0.020896 lr = 0.0000950
[0/25][3560/9765] Loss_D: 0.1435 Loss_G: 0.0524 Convergence: 0.1498 k= 0.021002 lr = 0.0000950
[0/25][3570/9765] Loss_D: 0.1403 Loss_G: 0.0438 Convergence: 0.1542 k= 0.021071 lr = 0.0000950
[0/25][3580/9765] Loss_D: 0.1478 Loss_G: 0.0569 Convergence: 0.1514 k= 0.021062 lr = 0.0000950
[0/25][3590/9765] Loss_D: 0.1509 Loss_G: 0.0502 Co

...

[22/25][3080/9765] Loss_D: 0.0984 Loss_G: 0.0395 Convergence: 0.0993 k= 0.018784 lr = 0.0000025
[22/25][3090/9765] Loss_D: 0.1059 Loss_G: 0.0383 Convergence: 0.1110 k= 0.018793 lr = 0.0000025
[22/25][3100/9765] Loss_D: 0.0931 Loss_G: 0.0374 Convergence: 0.0940 k= 0.018797 lr = 0.0000025
[22/25][3110/9765] Loss_D: 0.0918 Loss_G: 0.0361 Convergence: 0.0934 k= 0.018808 lr = 0.0000025
[22/25][3120/9765] Loss_D: 0.0977 Loss_G: 0.0365 Convergence: 0.1013 k= 0.018829 lr = 0.0000025
[22/25][3130/9765] Loss_D: 0.0914 Loss_G: 0.0378 Convergence: 0.0931 k= 0.018847 lr = 0.0000025
[22/25][3140/9765] Loss_D: 0.1005 Loss_G: 0.0384 Convergence: 0.1033 k= 0.018854 lr = 0.0000025
[22/25][3150/9765] Loss_D: 0.0885 Loss_G: 0.0381 Convergence: 0.0917 k= 0.018846 lr = 0.0000025
[22/25][3160/9765] Loss_D: 0.0896 Loss_G: 0.0395 Convergence: 0.0938 k= 0.018849 lr = 0.0000025
[22/25][3170/9765] Loss_D: 0.0966 Loss_G: 0.0408 Convergence: 0.0992 k= 0.018844 lr = 0.0000025
[22/25][3180/9765] Loss_D: 0.1137 Loss_G

[22/25][3940/9765] Loss_D: 0.0961 Loss_G: 0.0387 Convergence: 0.0968 k= 0.018813 lr = 0.0000025
[22/25][3950/9765] Loss_D: 0.0882 Loss_G: 0.0385 Convergence: 0.0919 k= 0.018810 lr = 0.0000025
[22/25][3960/9765] Loss_D: 0.0929 Loss_G: 0.0390 Convergence: 0.0951 k= 0.018820 lr = 0.0000025
[22/25][3970/9765] Loss_D: 0.0930 Loss_G: 0.0402 Convergence: 0.0964 k= 0.018816 lr = 0.0000025
[22/25][3980/9765] Loss_D: 0.0994 Loss_G: 0.0386 Convergence: 0.1016 k= 0.018818 lr = 0.0000025
[22/25][3990/9765] Loss_D: 0.1089 Loss_G: 0.0402 Convergence: 0.1133 k= 0.018826 lr = 0.0000025
[22/25][4000/9765] Loss_D: 0.0861 Loss_G: 0.0378 Convergence: 0.0899 k= 0.018831 lr = 0.0000025
[22/25][4010/9765] Loss_D: 0.0982 Loss_G: 0.0390 Convergence: 0.0994 k= 0.018836 lr = 0.0000025
[22/25][4020/9765] Loss_D: 0.0903 Loss_G: 0.0382 Convergence: 0.0928 k= 0.018839 lr = 0.0000025
[22/25][4030/9765] Loss_D: 0.1038 Loss_G: 0.0386 Convergence: 0.1077 k= 0.018847 lr = 0.0000025
[22/25][4040/9765] Loss_D: 0.0956 Loss_G

[22/25][4800/9765] Loss_D: 0.1067 Loss_G: 0.0399 Convergence: 0.1105 k= 0.018796 lr = 0.0000024
[22/25][4810/9765] Loss_D: 0.0976 Loss_G: 0.0388 Convergence: 0.0988 k= 0.018801 lr = 0.0000024
[22/25][4820/9765] Loss_D: 0.0940 Loss_G: 0.0390 Convergence: 0.0958 k= 0.018788 lr = 0.0000024
[22/25][4830/9765] Loss_D: 0.0949 Loss_G: 0.0389 Convergence: 0.0963 k= 0.018793 lr = 0.0000024
[22/25][4840/9765] Loss_D: 0.0999 Loss_G: 0.0386 Convergence: 0.1023 k= 0.018803 lr = 0.0000024
[22/25][4850/9765] Loss_D: 0.0906 Loss_G: 0.0369 Convergence: 0.0917 k= 0.018800 lr = 0.0000024
[22/25][4860/9765] Loss_D: 0.0974 Loss_G: 0.0387 Convergence: 0.0987 k= 0.018807 lr = 0.0000024
[22/25][4870/9765] Loss_D: 0.1022 Loss_G: 0.0376 Convergence: 0.1065 k= 0.018811 lr = 0.0000024
[22/25][4880/9765] Loss_D: 0.0962 Loss_G: 0.0381 Convergence: 0.0977 k= 0.018820 lr = 0.0000024
[22/25][4890/9765] Loss_D: 0.1055 Loss_G: 0.0388 Convergence: 0.1100 k= 0.018825 lr = 0.0000024
[22/25][4900/9765] Loss_D: 0.0933 Loss_G

[22/25][5660/9765] Loss_D: 0.0841 Loss_G: 0.0380 Convergence: 0.0889 k= 0.018856 lr = 0.0000024
[22/25][5670/9765] Loss_D: 0.0950 Loss_G: 0.0379 Convergence: 0.0962 k= 0.018857 lr = 0.0000024
[22/25][5680/9765] Loss_D: 0.0978 Loss_G: 0.0398 Convergence: 0.0989 k= 0.018863 lr = 0.0000024
[22/25][5690/9765] Loss_D: 0.0908 Loss_G: 0.0391 Convergence: 0.0940 k= 0.018855 lr = 0.0000024
[22/25][5700/9765] Loss_D: 0.1031 Loss_G: 0.0402 Convergence: 0.1053 k= 0.018856 lr = 0.0000024
[22/25][5710/9765] Loss_D: 0.0866 Loss_G: 0.0375 Convergence: 0.0899 k= 0.018861 lr = 0.0000024
[22/25][5720/9765] Loss_D: 0.0960 Loss_G: 0.0392 Convergence: 0.0972 k= 0.018863 lr = 0.0000024
[22/25][5730/9765] Loss_D: 0.0910 Loss_G: 0.0403 Convergence: 0.0954 k= 0.018859 lr = 0.0000024
[22/25][5740/9765] Loss_D: 0.0926 Loss_G: 0.0393 Convergence: 0.0953 k= 0.018847 lr = 0.0000024
[22/25][5750/9765] Loss_D: 0.0875 Loss_G: 0.0395 Convergence: 0.0925 k= 0.018849 lr = 0.0000024
[22/25][5760/9765] Loss_D: 0.0941 Loss_G

[22/25][6520/9765] Loss_D: 0.0948 Loss_G: 0.0384 Convergence: 0.0957 k= 0.018833 lr = 0.0000024
[22/25][6530/9765] Loss_D: 0.1013 Loss_G: 0.0388 Convergence: 0.1040 k= 0.018845 lr = 0.0000024
[22/25][6540/9765] Loss_D: 0.0975 Loss_G: 0.0408 Convergence: 0.0997 k= 0.018840 lr = 0.0000024
[22/25][6550/9765] Loss_D: 0.1040 Loss_G: 0.0392 Convergence: 0.1074 k= 0.018831 lr = 0.0000024
[22/25][6560/9765] Loss_D: 0.0945 Loss_G: 0.0385 Convergence: 0.0956 k= 0.018827 lr = 0.0000024
[22/25][6570/9765] Loss_D: 0.1063 Loss_G: 0.0393 Convergence: 0.1105 k= 0.018830 lr = 0.0000024
[22/25][6580/9765] Loss_D: 0.0964 Loss_G: 0.0377 Convergence: 0.0983 k= 0.018820 lr = 0.0000024
[22/25][6590/9765] Loss_D: 0.0975 Loss_G: 0.0383 Convergence: 0.0992 k= 0.018817 lr = 0.0000024
[22/25][6600/9765] Loss_D: 0.0940 Loss_G: 0.0387 Convergence: 0.0956 k= 0.018819 lr = 0.0000024
[22/25][6610/9765] Loss_D: 0.0945 Loss_G: 0.0387 Convergence: 0.0958 k= 0.018819 lr = 0.0000024
[22/25][6620/9765] Loss_D: 0.0926 Loss_G